# FLOYao.jl

A backend to efficiently simulate fermionic linear optics (FLO) circuits in [Yao.jl](https://github.com/QuantumBFS/Yao.jlhttps://github.com/QuantumBFS/Yao.jl) based on [Classical simulation of noninteracting-fermion quantum circuits](https://arxiv.org/abs/quant-ph/0108010)
and [Disorder-assisted error correction in Majorana chains](https://arxiv.org/abs/1108.3845).
FLO circuits are a class of quantum circuits that are closely related to non-interacting fermions 
and can be efficiently simulated on classical computers, similar to the way Clifford circuits
can be efficiently classically simulated, as is done in [YaoClifford.jl](https://github.com/QuantumBFS/YaoClifford.jl).
A quick introduction to fermionic linear optics circuits is found in [the Background section](#Background:-Fermionic-linear-optics-circuits) and a more in-depth introduction in e.g. the two papers linked above.

**Note**    
The markdown version of this README is automatically generated from `README.ipynb` and some 
of the hpyerlinks and math doesn't seem to play that well with githubs markdown parser. So you might want to have a look at `README.ipynb` with jupyter instead for a better reading experience.

## Contents
 - [Installation](#Installation)
 - [Basic usage](#Basic-usage)
 - [List of supported gates](#List-of-supported-gates)
 - [Example: Transverse field Ising model](#Example:-Transverse-Field-Ising-model)
 - [Adding support for your own gates](#Adding-support-for-your-own-gates)
 - [Background: Fermionic linear optics circuits](#Background:-Fermionic-linear-optics-circuits)
 - [Known restrictions](#Known-restrictions)

## Installation

 > By the time this will be in the julia general registry, it should be as easy as opening
 > a julia REPL and typing
 >  ```julia
 >   pkg> add FLOYao
 >  ```

But for now I suggest the following way: First open a standard terminal and type 
```bash
cd your_favorite_folder_for_code
git clone git@github.com:PhaseCraft/FLOYao.jl
```
and then open a julia REPL and type
```
pkg> dev your_favorite_folder_for_code/FLOYao.jl
```
which should make `FLOYao.jl` discoverable for your standard julia installation. 
Under linux the standard folder for julia packages under development is `/home/username/.julia/dev`

## Basic usage
The heart of `FLOYao` is the `MajoranaReg` register type, which efficiently represents a state that is a [FLO unitary](#Background:-Fermionic-linear-optics-circuits) applied to the vacuum state $|0⋯0⟩$

First import `Yao` and `FLOYao`:

In [1]:
using Yao, FLOYao

then build a (here somewhat arbitrary) circuit consisting only of [FLO gates](#Background:-Fermionic-linear-optics-circuits)

In [2]:
nq = 4
θ = π/8
circuit = chain(nq)

push!(circuit, put(nq, 3=>Rz(0.5)))

xxg1 = kron(nq, 1 => X, 2 => X)
rg = rot(xxg1, θ)
push!(circuit, rg)  

xxg2 = kron(nq, 2 => X, 3 => Z, 4 => X)
rg = rot(xxg2, θ)
push!(circuit, rg)  
push!(circuit, put(nq, 3=>Rz(0.5)))
push!(circuit, put(nq, 1=>Z))

xxg3 = kron(nq, 2 => X, 3 => X)
rg = rot(xxg3, θ)

circuit

nqubits: 4
chain
├─ put on (3)
│  └─ rot(Z, 0.5)
├─ rot(nqubits: 4
kron
├─ 1=>X
└─ 2=>X, 0.39269908169872414)
├─ rot(nqubits: 4
kron
├─ 2=>X
├─ 3=>Z
└─ 4=>X, 0.39269908169872414)
├─ put on (3)
│  └─ rot(Z, 0.5)
└─ put on (1)
   └─ Z


and define an observable that is a sum of squares of Majorana operators

In [3]:
hamiltonian = xxg1 + xxg2 + xxg3 + kron(nq, 2=>Z) + kron(nq, 3=>Z)

nqubits: 4
+
├─ kron
│  ├─ 1=>X
│  └─ 2=>X
├─ kron
│  ├─ 2=>X
│  ├─ 3=>Z
│  └─ 4=>X
├─ kron
│  ├─ 2=>X
│  └─ 3=>X
├─ kron
│  └─ 2=>Z
└─ kron
   └─ 3=>Z


and finally create a register in the computational zero state via

In [4]:
mreg = FLOYao.zero_state(nq)

MajoranaReg{Float64} with 4 qubits:
8×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

Other options to create registers are `FLOYao.product_state` and `FLOYao.zero_state_like`. `FLOYao.rand_state` is not implemented so far, because to do so I would need to rely on yet another dependency to create random orthogonal matrices.

Applying the circuit to the register works then exactly the same way as for a normal `ArrayReg` register:

In [5]:
apply(mreg, circuit)

MajoranaReg{Float64} with 4 qubits:
8×8 Matrix{Float64}:
 -1.0  -0.0       -0.0       -0.0       -0.0       -0.0       -0.0       -0.0
 -0.0  -0.92388    0.382683  -0.0       -0.0       -0.0       -0.0       -0.0
  0.0   0.382683   0.92388    0.0        0.0        0.0        0.0        0.0
  0.0   0.0        0.0        0.92388    0.0        0.0        0.382683   0.0
  0.0   0.0        0.0        0.0        0.540302   0.841471   0.0        0.0
  0.0   0.0        0.0        0.0       -0.841471   0.540302   0.0        0.0
  0.0   0.0        0.0       -0.382683   0.0        0.0        0.92388    0.0
  0.0   0.0        0.0        0.0        0.0        0.0        0.0        1.0

and the same goes for expectation values of observables

In [6]:
expval = expect(hamiltonian, mreg => circuit)

1.8535533905932737

or even gradients of these expectation values with respect to the circuit parameters

In [7]:
inδ, paramsδ = expect'(hamiltonian, mreg => circuit)

MajoranaReg{Float64}(4) => [0.0, -0.3535533905932738, -0.3535533905932738, 0.0]

It is also possible to sample from the state described by a `MajoranaReg`ister using `Yao`'s
`measure` or `measure!` functions. 

In [8]:
copyreg = copy(mreg) |> circuit
samples = measure(copyreg, nshots=10000)
samples[1:5]

5-element Vector{DitStr{2, 4, BigInt}}:
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎

and if you want to measure only a subset of qubits or change the ordering in
which the qubits are measured, this is also possible:

In [9]:
samples213 = measure(copyreg, [2, 1, 3],  nshots=10000)
samples213[1:5]

5-element Vector{DitStr{2, 3, BigInt}}:
 000 ₍₂₎
 000 ₍₂₎
 000 ₍₂₎
 000 ₍₂₎
 000 ₍₂₎

Just to check that this is all consistent with running a full wavefunction simulation, we can apply the same circuit on an `ArrayReg` and compare the expectation values, gradients and samples

In [10]:
areg = zero_state(nq)
expval_full = expect(hamiltonian, areg => circuit)
isapprox(expval_full, expval)

true

In [11]:
inδ_full, paramsδ_full = expect'(hamiltonian, areg => circuit)
isapprox(paramsδ, paramsδ_full)

true

In [12]:
copyareg = copy(areg) |> circuit
samples_full = measure(copyareg, nshots=10000)
samples213_full = measure(copyareg, [2, 1, 3], nshots=10000);

In [13]:
using StatsBase
println("Comparing the full countmaps:")
println("")
display(countmap(samples))
display(countmap(samples_full))
println("-----------------------------")

println("")
println("Comparing the countmaps on only a subset of qubits")
println("")
display(countmap(samples213))
display(countmap(samples213_full))
println("-----------------------------")

Comparing the full countmaps:



Dict{DitStr{2, 4, BigInt}, Int64} with 4 entries:
  0000 ₍₂₎ => 9246
  1010 ₍₂₎ => 372
  1001 ₍₂₎ => 15
  0011 ₍₂₎ => 367

Dict{DitStr{2, 4, Int64}, Int64} with 4 entries:
  0000 ₍₂₎ => 9236
  1010 ₍₂₎ => 349
  1001 ₍₂₎ => 18
  0011 ₍₂₎ => 397

-----------------------------

Comparing the countmaps on only a subset of qubits



Dict{DitStr{2, 3, BigInt}, Int64} with 4 entries:
  000 ₍₂₎ => 9298
  010 ₍₂₎ => 14
  011 ₍₂₎ => 352
  001 ₍₂₎ => 336

Dict{DitStr{2, 3, Int64}, Int64} with 4 entries:
  000 ₍₂₎ => 9269
  010 ₍₂₎ => 13
  011 ₍₂₎ => 362
  001 ₍₂₎ => 356

-----------------------------


which looks similar enough to trust that the samples come from the same probability distribution.

## List of supported gates

The following gates are FLO gates and supported by `FLOYao.jl`:

|  Gate | Comment |
|-------|---------|
|  `XGate`   | Together with `Y` the only gate that does not preserve fermionic parity |
|  `YGate`   |   See above  |
|  `ZGate`   |         |
|  `RotationGate{⋯,⋯,YGate}`  | The only single qubit rotation gate since $R_x(θ)γ_i R_x(-θ)$ is not a linear combination of Majorana operators for all Majorana operators. Similar for $R_y$ |
| `PauliKronBlock` | A kronecker product of Pauli operators s.t. that first and last operator are either $X$ or $Y$ and all operators in between are $Z$.  |
| `RotationGate{⋯,⋯,PauliKronBlock}` | A rotation gate with generator as in the last line. |
| `AbstractMatrix` | Unless the gate type is already explicitely implemented or know to not be a FLO gate, `FLOYao` will try to automatically convert the gate matrix in the qubit basis to a matrix in the Majorana basis. But note that this is fairly slow (although still poly-time instead of exp-time) |


If you want to add support to your own gates, read [this section](#Adding-support-for-your-own-gates) to learn how to do that.

## Example: VQE for the Transverse Field Ising model

For a more realistic use case, we have a look at VQE for the Transverse Field Ising model on a line whose Hamiltonian is given as 
$$
    H = J ∑_i^{L-1} X_i X_{i+1} + h ∑_i^L Z_i = U + T.
$$
As Ansatz circuits we use the Hamiltonian Variational Ansatz
$$
    U(\vec θ) = ∏_i^p e^{-iθ_{i,U} U} e^{-iθ_{i,T} T} 
$$
with the initial state being the groundstate of the TFIM at $J = 0$, so $|ψ_i⟩ = |0⋯0⟩$

In [14]:
L = 100 # this is far beyond what is possible with a full wavefunction simulation
J = 1.5
h = -1.
p = 10

U = map(1:L-1) do i
    J * kron(L, i => X, i+1 => X)
end |> sum

T = map(1:L) do i
    h * kron(L, i => Z)
end |> sum

hamiltonian = T + U

circuit = chain(L)
for _ in 1:p
    for i in 1:L-1
        push!(circuit, rot(kron(L, i => X, i+1 => X), 0.))
    end
    for i in 1:L
        push!(circuit, put(L, i => Rz(0.)))
    end
end
nparams = nparameters(circuit)
dispatch!(circuit, rand(nparams) ./ 100)

reg = FLOYao.zero_state(L);

now that we defined the hamiltonian, the ansatz circuit and the initial state we can perform
simple gradient descent on the energy expectation value to find an approximation to the
groundstate of $H$:

In [15]:
iterations = 100
γ = 2e-2

for i in 1:iterations
    _, grad = expect'(hamiltonian, reg => circuit)
    dispatch!(-, circuit, γ * grad)
    println("Iteration $i, energy = $(expect(hamiltonian, reg => circuit))")
end

Iteration 1, energy = -99.95871457112206
Iteration 2, energy = -100.0737851017236
Iteration 3, energy = -100.13846065965161
Iteration 4, energy = -100.22543194784396
Iteration 5, energy = -100.36326184737493
Iteration 6, energy = -100.58575290964693
Iteration 7, energy = -100.94505912687522
Iteration 8, energy = -101.52222210016981
Iteration 9, energy = -102.43888246281948
Iteration 10, energy = -103.86633439319165
Iteration 11, energy = -106.01978332663921
Iteration 12, energy = -109.11481237899028
Iteration 13, energy = -113.26222872880085
Iteration 14, energy = -118.31735608958387
Iteration 15, energy = -123.79059331565912
Iteration 16, energy = -128.9662708992411
Iteration 17, energy = -133.21639889958706
Iteration 18, energy = -136.26557650113082
Iteration 19, energy = -138.2072292249013
Iteration 20, energy = -139.33045667610835
Iteration 21, energy = -139.93774034241287
Iteration 22, energy = -140.25464996725245
Iteration 23, energy = -140.4205262188564
Iteration 24, energy = -1

## Adding support for your own gates

Natively, the only FLO gates that come already shipped with `Yao.jl` are the gates listed [here](#List-of-supported-gates). But there are many more FLO gates, one being for example the `FSWAP` gate which swaps to qubits while making sure to preserve the fermionic commutation relations

In [16]:
@const_gate FSWAP::ComplexF64 = [1 0 0 0; 0 0 1 0; 0 1 0 0; 0 0 0 -1]

If a gate defines a matrix representation, as we just did for the `FSWAP`gate, `FLOYao` supports them out of the box by manually checking if they are a FLO gate and then computing its matrix representation in the Majorana basis. But this method is fairly slow–though still poly-time and memory–compared to directly implementing `unsafe_apply!(::MajoranaReg, ::YourBlock)` and  `instruct!(::MajoranaReg, ::YourBlock)` and will warn you accordingly

In [17]:
nq = 4
fswap = put(nq, (1, 2) => FSWAP)
mreg = FLOYao.zero_state(nq)
mreg |> put(nq, 2 => X)
mreg |> fswap

┌ Warning: Calling manual instruct!(MajoranaReg{Float64}(4), ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im -1.0 + 0.0im], (1, 2)).
│ You can greatly speed up your FLO gates by exactly implementing unsafe_apply!()
│ and instruct!() for them. See FLOYao/src/instruct.jl and  FLOYao/src/apply_composite.jl
│ for how to do that.
└ @ FLOYao /home/yc20910/PhD/Work/code/FLOYao/src/instruct.jl:56


MajoranaReg{Float64} with 4 qubits:
8×8 Matrix{Float64}:
 -2.35415e-16  -4.12493e-16  -1.0          …   0.0   0.0   0.0   0.0
  2.46746e-16  -5.5708e-16   -1.26504e-16      0.0   0.0   0.0   0.0
 -1.0          -1.17708e-16   2.55988e-16      0.0   0.0   0.0   0.0
 -1.85286e-16  -1.0           2.44068e-16      0.0   0.0   0.0   0.0
 -0.0          -0.0          -0.0             -1.0  -0.0  -0.0  -0.0
 -0.0          -0.0          -0.0          …  -0.0  -1.0  -0.0  -0.0
 -0.0          -0.0          -0.0             -0.0  -0.0  -1.0  -0.0
 -0.0          -0.0          -0.0             -0.0  -0.0  -0.0  -1.0

Now, before we fix these warnings, let's see how long the current implementation takes:

In [18]:
using BenchmarkTools
using Suppressor # we don't want to get all the warnings when benchmarking
@benchmark @suppress apply!($mreg, $fswap)

BenchmarkTools.Trial: 6254 samples with 1 evaluation.
 Range (min … max):  713.850 μs …   3.865 ms  ┊ GC (min … max): 0.00% … 74.64%
 Time  (median):     759.798 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   794.402 μs ± 234.728 μs  ┊ GC (mean ± σ):  2.27% ±  6.20%

   ▃▆▃█▄▄▂                                                       
  ▃███████▇▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  714 μs           Histogram: frequency by time         1.24 ms <

 Memory estimate: 338.53 KiB, allocs estimate: 495.

To find out what the matrix representation of the `FSWAP` gate in the Majorana basis is, it is easiest to retrace what is happening inside `instruct!(::MajoranaReg, ::AbstractMatrix, locs)`
You can use

In [19]:
@which instruct!(mreg, mat(FSWAP), (1,2))

instruct!(reg::MajoranaReg, gate::AbstractMatrix, locs) in FLOYao at /home/yc20910/PhD/Work/code/FLOYao/src/instruct.jl:49

to find the location of the corresponding code. Now let's copy-paste what we found there:

In [20]:
W = FLOYao.qubit2majoranaevolution(Matrix(fswap.content), fswap.locs)

4×4 Matrix{Float64}:
 -2.35415e-16  -4.12493e-16  -1.0           0.0
  2.46746e-16  -5.5708e-16   -1.26504e-16  -1.0
 -1.0          -1.17708e-16   2.55988e-16  -2.38988e-16
 -1.85286e-16  -1.0           2.44068e-16   2.43374e-16

In [21]:
matlocs = 2*(fswap.locs[1]-1)+1:2(fswap.locs[end])

1:4

this matrix gets left-multiplied to the columns `1:4` in the last line of `FLOYao.majorana_unsafe_apply!(::MajoranaReg, ::PutBlock)`. So we can instead implement the action of an `FSWAP` gate on a `MajoranaReg` directly as follows:

In [22]:
function YaoBlocks.unsafe_apply!(reg::MajoranaReg, b::PutBlock{2,2,FSWAPGate})
    FLOYao.areconsecutive(b.locs) || throw(NonFLOException("FSWAP must act on consecutive qubits"))
    instruct!(reg, Val(:FSWAP), b.locs)
end

function Yao.instruct!(reg::MajoranaReg, ::Val{:FSWAP}, locs::Tuple)
    i1, i2 = locs
    row1, row2 = reg.state[2i1-1,:], reg.state[2i1,:]
    row3, row4 = reg.state[2i2-1,:], reg.state[2i2,:]
    reg.state[2i1-1,:] .=  .-row3
    reg.state[2i1,:] .=  .-row4
    reg.state[2i2-1,:] .=  .-row1
    reg.state[2i2,:] .=  .-row2
    return reg
end

In [23]:
@benchmark apply!($mreg, $fswap)

BenchmarkTools.Trial: 10000 samples with 585 evaluations.
 Range (min … max):  188.162 ns …   4.742 μs  ┊ GC (min … max): 0.00% … 92.38%
 Time  (median):     213.903 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   249.819 ns ± 296.990 ns  ┊ GC (mean ± σ):  9.48% ±  7.55%

     ▅▄█▆▅▆▆▅▄▄▄▃▂▁▁                               ▁▁▁▁         ▂
  ▃▅█████████████████▇▆▅▆▄▄▄▅▃▄▅▅▅▃▄▅▅▅▆▆▆▆▆▇▇▇▇████████████▇▇▆ █
  188 ns        Histogram: log(frequency) by time        388 ns <

 Memory estimate: 512 bytes, allocs estimate: 4.

Now, that is quite a significant speedup!

## Background: Fermionic linear optics circuits

This section is here, more to fix the convention of
[Jordan-Wigner transform](https://en.wikipedia.org/wiki/Jordan%E2%80%93Wigner_transformation)
and [Majorana operators](https://en.wikipedia.org/wiki/Majorana_fermion) that we use here,
and less to explain the full theory behind those. For the latter, we, once again, recommend [Classical simulation of noninteracting-fermion quantum circuits](https://arxiv.org/abs/quant-ph/0108010).

We define the Majorana operators $γ_i$ via 
$$
    γ_{2i-1} = ∏_{j=1}^{i-1} (-Z_j) X_i
$$
and
$$
    γ_{2i} = ∏_{j=1}^{i-1} (-Z_j) Y_i.
$$
This implies the normal fermionic creation and annihilation operators are given by
$$
    c_j = \frac{1}{2} (γ_{2j-1} + iγ_{2j})
    \quad \textrm{and} \quad
    c_j^† = \frac{1}{2} (γ_{2j-1} - iγ_{2j})
$$
and products of two Majorana operators are of the form
$$
    σ_i \left(∏_{i<j<k} Z_k \right) σ_k
    \quad \textrm{or} \quad
    Z_i
$$
with $σ_i, σ_k ∈ \{X, Y\}$.

Any unitary that takes all Majorana operators to a linear combination of Majorana operators
under conjugation, i.e. that satisfies
$$
    U γ_i U^† = R_i^j γ_j
$$
with some $R ∈ O(2n)$ is a FLO unitary. In particular, if a unitary is of the form 
$$
    U = e^{-iθH}
$$
with 
$$
    H = \frac{i}{4} \sum_{i,j} H^{ij} γ_i γ_j
$$
it is a FLO unitary with even $R ∈ SO(2n)$.

But note, that not all FLO unitaries are of that form. For example $X_i$ is also a FLO 
gate since it either commutes or anti-commutes with all Majorana operators, but the associated
matrix $R$ always has determinant $-1$.

Calculating the expectation values of hamiltonians like the one above when evolving the 
vacuum state with FLO circuits is efficiently possible. First evolve the 
Hamiltonian in the Heisenber picture to
$$
    UHU^† = \frac{i}{4} R^{m}_{i} R^{n}_{j} H^{ij} γ_{m} γ_{n} 
           =: \frac{i}{4} \tilde H^{mn} γ_{m} γ_{n}.
$$
and then compute the expectation value
$$
\begin{aligned}
    ⟨ψ|UHU^†|ψ⟩ &= \frac{i}{4} \tilde H^{mn} ⟨Ω|γ_{m} γ_{n}|Ω⟩ \\
                &= - \frac{1}{2} ∑_{i} \tilde H^{2i-1,2i} \\
                &= - \frac{1}{2} ∑_{i>k} R^{2i-1}_{m} R^{2i}_{n} H^{mn} \\
\end{aligned}.
$$
From the first to second line one needs to carefully think which of the 
$⟨Ω|γ_{m} γ_{n}|Ω⟩$ are zero and which cancel each other out due to the anti-symmetry of $H^{mn}$.

## Known restrictions

###  Expectation values of higher order observables
So far, `FLOYao` only supports expectation values of observables that are sums of squares of 
Majorana operators. But in general, one can use [Wick's theorem](https://en.wikipedia.org/wiki/Wick%27s_theorem) to calculate the expectation values of expressions of the form 
$$
    ⟨ψ_i|O|ψ_i⟩
$$
where $|ψ_i⟩$ is a computational basis state and $O$ a string of Majorana operators and thus, using linearity of the expectation value, it is possible to efficiently calculate the expectation value of any observable that can be expanded in a sum of polynomially (in the number of qubits) many products of Majorana operators. (See also [Classical simulation of noninteracting-fermion quantum circuits](https://arxiv.org/abs/quant-ph/0108010) again for details). If you need expectation values of higher order (in the number of Majorana operators involved) observables, feel free to open a pull request!

### "Hidden" FLO circuits
`Yao.jl` allows to express the same circuit in different forms. E.g. 
```julia
    chain(nq, put(1 => X), put(2 => X))
```
and
```julia
    kron(nq, 1 => X, 2 => X)
```
represent the same FLO circuit, but only the latter will be recognised as such. Similarly 
```julia
     kron(nq, 1 => X, 2 => X, 3 => X, 4 => X)
```
and
```julia
     chain(nq, kron(1 => X, 2 => X), kron(3 => X, 4 => X))
```
represent the same FLO circuit but only the latter will be recognised as such. This is because

 - We don't check if a whole `ChainBlock` is a FLO circuit, even if its single
   gates are not. Instead a `ChainBlock` is applied gate by gate, each of which
   has to be a FLO gate.
 - For `KronBlock`s we first try if each factor is a FLO gate and then if the whole
   block altogether is of the form  `kron(nq, i => σ1, i+1 => Z, ⋯, i-1 => Z, j => σ2)`
   with `σ1, σ2 ∈ [X, Y]`.

If you run into a case that is a FLO circuit / gate but not recognised as such please open an issue or even pull request.
